In this notebook I will show how to work with real-time data that updates not only the column values (relative to time), but also adds new rows to existing table. I will use a .csv file that contains information from my YouTube channel videos. I've extracted that CSV file from my 'youtube-api-tutorial' repo. 

In [1]:
#First of all, videos on my channel aren't really type of content that someone will see on daily basis, 
#so in expecation things such as view_count, like_count etc. on average will stay the same.
#I will manually update values of some existing columns, and also imitate the scenario of having a new video(s) by appending row(s) and checking if my code works based on that. 

In [2]:
#Installing the necessary libraries
!pip install mysql-connector-python

In [3]:
#Importing libraries
import mysql.connector
from mysql.connector.constants import ClientFlag
import pandas as pd

In [4]:
#Let's upload our .csv file that contains informations of my videos:
df = pd.read_csv('my_youtubeWo.csv')
df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,-2i4xmW6u2A,ZHU - In the Morning* [Short Cover],2018-09-23,288,11,0
1,avbZDwi4VTM,Amra Mandžukić #4 //BIH Women&#39;s National V...,2018-05-17,808,13,0
2,TWkT6ovxAtQ,Animal Farm (BOSNIA Edition),2018-03-01,127,4,0
3,Bj0CMyhLDRU,Poklon Čani,2018-02-10,113,6,0
4,neGDJyEpkVw,žeks from skugric $$ thug life $$,2018-01-30,211,4,0


In [5]:
def connect_to_db(user, password, host_ip):
  '''
      Connecting to cloud database.
  '''
  cnxn = None
  config = {
    'user': user,
    'password': password,
    'host': host_ip,
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'ssl/server-ca.pem',
    'ssl_cert': 'ssl/client-cert.pem',
    'ssl_key': 'ssl/client-key.pem'
            }


  try:  
       cnxn = mysql.connector.connect(**config)
  except:
       print('Error! Please check if you put the right information into given variables (password, user etc.).')
  else:
      print('You are sucessfully connected to the Cloud instance!')
    
  return cnxn

In [6]:
conn_db = connect_to_db(user = 'Neo', password = 'iDontThinkSo', host_ip = '992.1199.20.202') 

You are sucessfully connected to the Cloud instance!


In [7]:
conn_db.is_connected() #It connects us to the Cloud server

True

In [8]:
#Now let's create a database 
cursor = conn_db.cursor()
cursor.execute('CREATE DATABASE IF NOT EXISTS my_youtube')

#We are going to disconnect from the Cloud SQL instance and reconnect to the specific database we made 'my_youtube'
conn_db.close()
conn_db.is_connected()

False

In [9]:
#vars(conn_db)
conn_db._database = 'my_youtube'
print(conn_db._database)

my_youtube


In [10]:
conn_db.connect()
print(conn_db.is_connected(), conn_db._database)

True my_youtube


In [11]:
##Now let's create a table in our database
df.head(1)

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,-2i4xmW6u2A,ZHU - In the Morning* [Short Cover],2018-09-23,288,11,0


In [12]:
df.dtypes

video_id         object
video_title      object
upload_date      object
view_count        int64
like_count        int64
comment_count     int64
dtype: object

In [13]:
#We are going to set video_id as our Primary key. On other columns we wont allow NULL values
#I'm not going to wrap this into function since each table has different columns and datatypes ;)

In [15]:
create_table_query = ('''CREATE TABLE IF NOT EXISTS yt_channel_videos (
                          video_id VARCHAR(255) PRIMARY KEY,
                          video_title TEXT NOT NULL,
                          upload_date DATE NOT NULL,
                          view_count INTEGER NOT NULL,
                          like_count INTEGER NOT NULL,
                          comment_count INTEGER NOT NULL)''')

cursor.execute(operation = create_table_query)

conn_db.commit()

Before we even start populating the remote table we will first design the implementation of update/insert functions. The main idea will revolve around 'one video per update/insert'. We will first check if that video exists in our table, and if so we will update the columns on the fly. If not, we will create a separate dataframe to hold new rows, and depending on the schedule we intend to program that separate dataframe might contain huge amounts of rows! The idea of importing them all at once into our remote table would possibly have performance/memory issues, and hence we will do it 'per video'.

In [16]:
#Here is the overview of the code logic
'''for i, row in df.iterrows():
  if check_if_video_exists():
    update_row()
  else:
    append(row)'''

'for i, row in df.iterrows():\n  if check_if_video_exists():\n    update_row()\n  else:\n    append(row)'

In [17]:
#Checking if video exists
def check_if_video_exists(cursor, video_id):
  query = ('''SELECT video_id FROM yt_channel_videos WHERE video_id = %s''')
  cursor.execute(query, (video_id,))
  return cursor.fetchone() is not None #If there is a row fetchone() will return it, if not it will return None

In [18]:
#Update rows
def update_row(cursor, video_id, video_title, view_count, like_count, comment_count):
  query = ("""UPDATE yt_channel_videos
            SET video_title = %s, view_count = %s, like_count = %s, comment_count = %s
            WHERE video_id = %s;""")
  vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
  cursor.execute(query, vars_to_update)

In [19]:
tmp_df = pd.DataFrame(columns = list(df.columns))
tmp_df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count


In [20]:
def update_db(cursor, df):  
 tmp_df = pd.DataFrame(columns = list(df.columns)) #Temporary dataframe for storing new videos that aren't in a database
 for i, row in df.iterrows():
       if check_if_video_exists(cursor, row['video_id']):
         update_row(cursor, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['comment_count'])
       else:
          tmp_df = tmp_df.append(row)
 return tmp_df 

In [21]:
#Insert new rows
def insert_new_rows(cursor, video_id, video_title, upload_date, view_count, like_count, comment_count):
  query = ("""INSERT INTO yt_channel_videos (video_id, video_title, upload_date, view_count, like_count, comment_count)
            VALUES(%s, %s, %s, %s, %s, %s)""")
  row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
  cursor.execute(query, row_to_insert)

In [22]:
#Inserting new rows (row by row) into database
def append_df_to_db(cursor, df):
  for i, row in df.iterrows():
    insert_new_rows(cursor, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

In [23]:
new_df = update_db(cursor, df)
conn_db.commit()

In [24]:
new_df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,-2i4xmW6u2A,ZHU - In the Morning* [Short Cover],2018-09-23,288,11,0
1,avbZDwi4VTM,Amra Mandžukić #4 //BIH Women&#39;s National V...,2018-05-17,808,13,0
2,TWkT6ovxAtQ,Animal Farm (BOSNIA Edition),2018-03-01,127,4,0
3,Bj0CMyhLDRU,Poklon Čani,2018-02-10,113,6,0
4,neGDJyEpkVw,žeks from skugric $$ thug life $$,2018-01-30,211,4,0
5,CvRncH8rdPc,"Riblja Čorba - Bože, koliko je volim (COVER)",2017-10-02,491,14,2
6,VgB6hCpFong,Šoferska je tuga pregolema (Guitar),2017-04-13,523,5,8
7,dUMmMRKPek4,Scientific method fromBosnia be like...,2017-02-20,190,3,0
8,lZvB18OSDe8,Intro 1,2017-02-18,36,0,0
9,-fVxmrQeTBA,Test 1,2017-02-15,76,1,4


In [25]:
append_df_to_db(cursor, new_df)
conn_db.commit()

In [26]:
#Let's check if that works!
query = '''SELECT * FROM yt_channel_videos;'''
cursor.execute(query)
output = cursor.fetchall()
for row in output:
  print(row)

('-2i4xmW6u2A', 'ZHU - In the Morning* [Short Cover]', datetime.date(2018, 9, 23), 288, 11, 0)
('-fVxmrQeTBA', 'Test 1', datetime.date(2017, 2, 15), 76, 1, 4)
('-zy2pXzqwCM', 'Zabranjeno Pušenje - Ženi Nam Se Vukota //Solo (Cover)', datetime.date(2017, 1, 21), 3839, 41, 3)
('1kFw5iGUxSU', 'REPILF - 1312 NIJANSI CRNE (SPOT)', datetime.date(2016, 11, 26), 31537, 322, 32)
('4mVlcq8BHJk', 'dark cloud - broken promise (guitar cover)', datetime.date(2016, 8, 1), 335, 9, 0)
('avbZDwi4VTM', 'Amra Mandžukić #4 //BIH Women&#39;s National Volleyball Team', datetime.date(2018, 5, 17), 808, 13, 0)
('Bj0CMyhLDRU', 'Poklon Čani', datetime.date(2018, 2, 10), 113, 6, 0)
('CvRncH8rdPc', 'Riblja Čorba - Bože, koliko je volim (COVER)', datetime.date(2017, 10, 2), 491, 14, 2)
('dUMmMRKPek4', 'Scientific method fromBosnia be like...', datetime.date(2017, 2, 20), 190, 3, 0)
('EnqCrg8J0YU', 'snimanje karakteristika tranzistora.', datetime.date(2016, 7, 12), 252, 4, 0)
('fD_TvzuH_vE', 'merrie melodies opening 

In [27]:
len(output) == len(new_df) #There we go ;)

True

In [28]:
#Let's see if the update function works. 
#Now, I'm going to make a dataframe with a synthetic data such that some of the columns from the existing rows are changed (view_counts etc.),
#and I'm also going to add some new videos with random stuff that mimic this dataset, since I'm not that YouTube popular ;)

df_syn = df.copy() #Synthetic dataset
df_syn.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,-2i4xmW6u2A,ZHU - In the Morning* [Short Cover],2018-09-23,288,11,0
1,avbZDwi4VTM,Amra Mandžukić #4 //BIH Women&#39;s National V...,2018-05-17,808,13,0
2,TWkT6ovxAtQ,Animal Farm (BOSNIA Edition),2018-03-01,127,4,0
3,Bj0CMyhLDRU,Poklon Čani,2018-02-10,113,6,0
4,neGDJyEpkVw,žeks from skugric $$ thug life $$,2018-01-30,211,4,0


In [29]:
#View count must be more than existing and hence: view_count = view_count + randomint()
df_syn.at[1, 'view_count']

808

In [30]:
#Generating random integers

from random import seed, randint
#Setting a random seed is not *that necessary* because we want our values to be random, and possibly each time different.
#The logic remains the same, the p-value is still 0.05 :P 
seed(42) 
for _ in range(10):
	value = randint(1000, 100000) #I'm forecasting to be so popular on YouTube :P
	print(value)

84810
15592
4278
98196
37048
33098
30256
19289
97530
14434


In [31]:
#######VIEW COUNTS#######

In [32]:
#Values of view_count before:
df_syn.at[1, 'view_count'], df_syn.at[5, 'view_count'], df_syn.at[8, 'view_count']

(808, 491, 36)

In [33]:
#We are going to change just a few of a videos
df_syn.at[1, 'view_count'] = df_syn.at[1, 'view_count'] + randint(1000,10000)
df_syn.at[5, 'view_count'] = df_syn.at[5, 'view_count'] + randint(1000,10000)
df_syn.at[8, 'view_count'] = df_syn.at[8, 'view_count'] + randint(1000,10000)

In [34]:
#Values of view_count after:
df_syn.at[1, 'view_count'], df_syn.at[5, 'view_count'], df_syn.at[8, 'view_count']

(10743, 2915, 7948)

In [35]:
#######LIKE COUNTS#######

In [36]:
#Values of like_count before 
df_syn.at[2, 'like_count'], df_syn.at[4, 'like_count'], df_syn.at[9, 'like_count']

(4, 4, 1)

In [37]:
#Values of view_count before:
df_syn.at[2, 'like_count'] = df_syn.at[2, 'like_count'] + randint(10,100)
df_syn.at[4, 'like_count'] = df_syn.at[4, 'like_count'] + randint(10,100)
df_syn.at[9, 'like_count'] = df_syn.at[9, 'like_count'] + randint(10,100)

In [38]:
#Values of like_count after
df_syn.at[2, 'like_count'], df_syn.at[4, 'like_count'], df_syn.at[9, 'like_count']

(18, 17, 22)

In [39]:
#######NEW VIDEOS#######

In [40]:
##RANDOM VIDEO IDS
from random import choice
from string import ascii_letters as al
length_of_random_string = 11 #Since all of the video_id's are of length 11

random_string = "".join(choice(al) for i in range(length_of_random_string))

print(random_string)

noGMbJmTPSI


In [41]:
len('jnWvgfygwwM')

11

#Probability of having IDs same as ones I already have?
Given that video_id is a string of length 11 and contains set of ascii_letters where len(ascii_letters) = 128 (128 unique chars) it would mean that there are 128^11=1.5111573e+23 unique possibilities. Probability is pretty much num_of_videos/128^11. Probability of having collision tends to 0, BUT there is still a possibility. ;) 

In [42]:
##RANDOM DATES
!pip install Faker

In [43]:
#We are lucky enough that dates are sorted in descending order ;)
df_syn.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,-2i4xmW6u2A,ZHU - In the Morning* [Short Cover],2018-09-23,288,11,0
1,avbZDwi4VTM,Amra Mandžukić #4 //BIH Women&#39;s National V...,2018-05-17,10743,13,0
2,TWkT6ovxAtQ,Animal Farm (BOSNIA Edition),2018-03-01,127,18,0
3,Bj0CMyhLDRU,Poklon Čani,2018-02-10,113,6,0
4,neGDJyEpkVw,žeks from skugric $$ thug life $$,2018-01-30,211,17,0


In [44]:
type(df_syn.at[0, 'upload_date'])

str

In [45]:
###### NEW ROWS ######

In [46]:
#I'm going to use Faker library so that I can generate random dates
#Using the basic common sense we can conclude this logic: This dataframe contains all the videos that I ever uploaded, so 
#I can't add a video that is older than todays date and earlier than the latest video I've uploaded
#Well, since YouTube API collects only Public videos, there might be a possibility that I've made some videos Private which might be 'younger' or 'equal'
#than my latest video upload and older than date of my account creation. 

from faker import Faker
import datetime
fake = Faker()

#Date today
today_date = datetime.date.today()
today_year = int(today_date.strftime('%Y'))
today_month = int(today_date.strftime('%m'))
today_day = int(today_date.strftime('%d'))

#Latest video date
latest_date = df_syn.at[0, 'upload_date'].split('-')
ld_year = int(df_syn.at[0, 'upload_date'].split('-')[0])
ld_month = int(df_syn.at[0, 'upload_date'].split('-')[1])
ld_day = int(df_syn.at[0, 'upload_date'].split('-')[2])


start_date = datetime.date(year = ld_year, month = ld_month, day = ld_day)
end_date = datetime.date(year = today_year, month = today_month, day = today_day)
date_in_between = fake.date_between(start_date = start_date, end_date = end_date)

print(f'Start date: {start_date}; End date: {end_date}; Date in between: {date_in_between}.')

Start date: 2018-09-23; End date: 2022-02-10; Date in between: 2019-05-03.


In [47]:
#This seems good!

In [81]:
%pprint

Pretty printing has been turned ON


In [49]:
#metoo ^

In [50]:
##RANDOM VIDEO TITLES

#Let's train a Youtube Video Title Generator using LSTMs... NO!

In [51]:
##DETERMINISTIC VIDEO TITLES
video_1_title = 'How to train a Youtube Video Title Generator using LSTMs'
video_2_title = "I'm so lazy to train a Youtube Video Title Generator using LSTMs (Part 1)"
video_3_title = "I'm so lazy to train a Youtube Video Title Generator using LSTMs (Part 2)"

In [52]:
fake.date_between(start_date = start_date, end_date = end_date)

datetime.date(2021, 1, 18)

In [53]:
#Fake video dates
video_dates = []
for i in range(3):
  video_dates.append(fake.date_between(start_date = start_date, end_date = end_date))
print(video_dates)

[datetime.date(2021, 12, 11), datetime.date(2021, 10, 22), datetime.date(2020, 8, 22)]


In [54]:
#Fake dates
video_1_date = str(video_dates[0])
video_2_date = str(video_dates[1])
video_3_date = str(video_dates[2])

In [55]:
#Random Video IDs
random_string_1 = "".join(choice(al) for i in range(length_of_random_string))
random_string_2 = "".join(choice(al) for i in range(length_of_random_string))
random_string_3 = "".join(choice(al) for i in range(length_of_random_string))

In [56]:
print(video_1_date, video_2_date)

2021-12-11 2021-10-22


In [57]:
#Three new videos
new_row_1 = {'video_id':random_string_1,
             'video_title': video_1_title,
             'upload_date': video_1_date, 
              'view_count': randint(1000,10000), 
              'like_count': randint(10,100), 
              'comment_count': randint(69,250)}

new_row_2 = {'video_id':random_string_2,
             'video_title': video_2_title,
             'upload_date': video_2_date, 
              'view_count': randint(1000,10000), 
              'like_count': randint(10,100), 
              'comment_count': randint(69,250)}

new_row_3 = {'video_id':random_string_3,
             'video_title': video_3_title,
             'upload_date': video_3_date, 
              'view_count': randint(1000,10000), 
              'like_count': randint(10,100), 
              'comment_count': randint(69,250)}


In [58]:
temp_df = pd.DataFrame(columns = list(df_syn.columns))
temp_df

,video_id,video_title,upload_date,view_count,like_count,comment_count


In [59]:
temp_df = temp_df.append(new_row_1, ignore_index = True)
temp_df = temp_df.append(new_row_2, ignore_index = True)
temp_df = temp_df.append(new_row_3, ignore_index = True)

In [60]:
temp_df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,AoCLrZaWZkS,How to train a Youtube Video Title Generator u...,2021-12-11,2291,80,144
1,BvrjnWvgfyg,I'm so lazy to train a Youtube Video Title Gen...,2021-10-22,6925,83,118
2,wwMqZcUDIhy,I'm so lazy to train a Youtube Video Title Gen...,2020-08-22,2139,15,238


In [61]:
#Nice! Let's check if our insert and update functionality works!
#First let's see how our table looks before adding these videos:

In [62]:
#Before
query = '''SELECT * FROM yt_channel_videos;'''
cursor.execute(query)
output = cursor.fetchall()
for row in output:
  print(row)

('-2i4xmW6u2A', 'ZHU - In the Morning* [Short Cover]', datetime.date(2018, 9, 23), 288, 11, 0)
('-fVxmrQeTBA', 'Test 1', datetime.date(2017, 2, 15), 76, 1, 4)
('-zy2pXzqwCM', 'Zabranjeno Pušenje - Ženi Nam Se Vukota //Solo (Cover)', datetime.date(2017, 1, 21), 3839, 41, 3)
('1kFw5iGUxSU', 'REPILF - 1312 NIJANSI CRNE (SPOT)', datetime.date(2016, 11, 26), 31537, 322, 32)
('4mVlcq8BHJk', 'dark cloud - broken promise (guitar cover)', datetime.date(2016, 8, 1), 335, 9, 0)
('avbZDwi4VTM', 'Amra Mandžukić #4 //BIH Women&#39;s National Volleyball Team', datetime.date(2018, 5, 17), 808, 13, 0)
('Bj0CMyhLDRU', 'Poklon Čani', datetime.date(2018, 2, 10), 113, 6, 0)
('CvRncH8rdPc', 'Riblja Čorba - Bože, koliko je volim (COVER)', datetime.date(2017, 10, 2), 491, 14, 2)
('dUMmMRKPek4', 'Scientific method fromBosnia be like...', datetime.date(2017, 2, 20), 190, 3, 0)
('EnqCrg8J0YU', 'snimanje karakteristika tranzistora.', datetime.date(2016, 7, 12), 252, 4, 0)
('fD_TvzuH_vE', 'merrie melodies opening 

In [63]:
#First we are going to insert new rows into our remote table

In [64]:
new_columns_df = update_db(cursor, temp_df)
new_columns_df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,AoCLrZaWZkS,How to train a Youtube Video Title Generator u...,2021-12-11,2291,80,144
1,BvrjnWvgfyg,I'm so lazy to train a Youtube Video Title Gen...,2021-10-22,6925,83,118
2,wwMqZcUDIhy,I'm so lazy to train a Youtube Video Title Gen...,2020-08-22,2139,15,238


In [65]:
append_df_to_db(cursor, new_columns_df)
db_conn.commit()

In [66]:
#After
query = '''SELECT * FROM yt_channel_videos;'''
cursor.execute(query)
output = cursor.fetchall()
for row in output:
  print(row)

('-2i4xmW6u2A', 'ZHU - In the Morning* [Short Cover]', datetime.date(2018, 9, 23), 288, 11, 0)
('-fVxmrQeTBA', 'Test 1', datetime.date(2017, 2, 15), 76, 1, 4)
('-zy2pXzqwCM', 'Zabranjeno Pušenje - Ženi Nam Se Vukota //Solo (Cover)', datetime.date(2017, 1, 21), 3839, 41, 3)
('1kFw5iGUxSU', 'REPILF - 1312 NIJANSI CRNE (SPOT)', datetime.date(2016, 11, 26), 31537, 322, 32)
('4mVlcq8BHJk', 'dark cloud - broken promise (guitar cover)', datetime.date(2016, 8, 1), 335, 9, 0)
('AoCLrZaWZkS', 'How to train a Youtube Video Title Generator using LSTMs', datetime.date(2021, 12, 11), 2291, 80, 144)
('avbZDwi4VTM', 'Amra Mandžukić #4 //BIH Women&#39;s National Volleyball Team', datetime.date(2018, 5, 17), 808, 13, 0)
('Bj0CMyhLDRU', 'Poklon Čani', datetime.date(2018, 2, 10), 113, 6, 0)
('BvrjnWvgfyg', "I'm so lazy to train a Youtube Video Title Generator using LSTMs (Part 1)", datetime.date(2021, 10, 22), 6925, 83, 118)
('CvRncH8rdPc', 'Riblja Čorba - Bože, koliko je volim (COVER)', datetime.date(201

In [67]:
#After
query = '''SELECT * FROM yt_channel_videos WHERE video_id = 'avbZDwi4VTM';'''
cursor.execute(query)
output = cursor.fetchall()
for row in output:
  print(row)

('avbZDwi4VTM', 'Amra Mandžukić #4 //BIH Women&#39;s National Volleyball Team', datetime.date(2018, 5, 17), 808, 13, 0)


#Updating existing columns with new data

In [68]:
#We are going to first change data types of synthetic dataframe to match the ones we are already using

In [69]:
df_syn.dtypes

video_id         object
video_title      object
upload_date      object
view_count        int64
like_count        int64
comment_count     int64
dtype: object

In [70]:
new_df.dtypes

video_id         object
video_title      object
upload_date      object
view_count       object
like_count       object
comment_count    object
dtype: object

In [71]:
df_syn['view_count'] = df_syn.view_count.astype(str)
df_syn['like_count'] = df_syn.like_count.astype(str)
df_syn['comment_count'] = df_syn.comment_count.astype(str)

In [72]:
df_syn.dtypes == new_df.dtypes

video_id         True
video_title      True
upload_date      True
view_count       True
like_count       True
comment_count    True
dtype: bool

In [73]:
#That looks better!

In [74]:
#Let's query the results before we update it so we can be sure that the values will be changed
query = '''SELECT * FROM yt_channel_videos;'''
cursor.execute(query)
output = cursor.fetchall()
for row in output:
  print(row)

('-2i4xmW6u2A', 'ZHU - In the Morning* [Short Cover]', datetime.date(2018, 9, 23), 288, 11, 0)
('-fVxmrQeTBA', 'Test 1', datetime.date(2017, 2, 15), 76, 1, 4)
('-zy2pXzqwCM', 'Zabranjeno Pušenje - Ženi Nam Se Vukota //Solo (Cover)', datetime.date(2017, 1, 21), 3839, 41, 3)
('1kFw5iGUxSU', 'REPILF - 1312 NIJANSI CRNE (SPOT)', datetime.date(2016, 11, 26), 31537, 322, 32)
('4mVlcq8BHJk', 'dark cloud - broken promise (guitar cover)', datetime.date(2016, 8, 1), 335, 9, 0)
('AoCLrZaWZkS', 'How to train a Youtube Video Title Generator using LSTMs', datetime.date(2021, 12, 11), 2291, 80, 144)
('avbZDwi4VTM', 'Amra Mandžukić #4 //BIH Women&#39;s National Volleyball Team', datetime.date(2018, 5, 17), 808, 13, 0)
('Bj0CMyhLDRU', 'Poklon Čani', datetime.date(2018, 2, 10), 113, 6, 0)
('BvrjnWvgfyg', "I'm so lazy to train a Youtube Video Title Generator using LSTMs (Part 1)", datetime.date(2021, 10, 22), 6925, 83, 118)
('CvRncH8rdPc', 'Riblja Čorba - Bože, koliko je volim (COVER)', datetime.date(201

In [75]:
#One of the videos that will be updated is one with ID = 'avbZDwi4VTM' so we will check it before and after 
query = '''SELECT * FROM yt_channel_videos WHERE video_id = 'avbZDwi4VTM';'''
cursor.execute(query)
output = cursor.fetchone()
for row in output:
  print(row)

avbZDwi4VTM
Amra Mandžukić #4 //BIH Women&#39;s National Volleyball Team
2018-05-17
808
13
0


In [76]:
updated_columns_df = update_db(cursor, df_syn)

In [80]:
append_df_to_db(cursor, updated_columns_df)
conn_db.commit()

In [78]:
#Let's check if we can eyeball that everything is updated
query = '''SELECT * FROM yt_channel_videos;'''
cursor.execute(query)
output = cursor.fetchall()
for row in output:
  print(row)

('-2i4xmW6u2A', 'ZHU - In the Morning* [Short Cover]', datetime.date(2018, 9, 23), 288, 11, 0)
('-fVxmrQeTBA', 'Test 1', datetime.date(2017, 2, 15), 76, 22, 4)
('-zy2pXzqwCM', 'Zabranjeno Pušenje - Ženi Nam Se Vukota //Solo (Cover)', datetime.date(2017, 1, 21), 3839, 41, 3)
('1kFw5iGUxSU', 'REPILF - 1312 NIJANSI CRNE (SPOT)', datetime.date(2016, 11, 26), 31537, 322, 32)
('4mVlcq8BHJk', 'dark cloud - broken promise (guitar cover)', datetime.date(2016, 8, 1), 335, 9, 0)
('AoCLrZaWZkS', 'How to train a Youtube Video Title Generator using LSTMs', datetime.date(2021, 12, 11), 2291, 80, 144)
('avbZDwi4VTM', 'Amra Mandžukić #4 //BIH Women&#39;s National Volleyball Team', datetime.date(2018, 5, 17), 10743, 13, 0)
('Bj0CMyhLDRU', 'Poklon Čani', datetime.date(2018, 2, 10), 113, 6, 0)
('BvrjnWvgfyg', "I'm so lazy to train a Youtube Video Title Generator using LSTMs (Part 1)", datetime.date(2021, 10, 22), 6925, 83, 118)
('CvRncH8rdPc', 'Riblja Čorba - Bože, koliko je volim (COVER)', datetime.date(

In [79]:
#Here is the same video 
query = '''SELECT * FROM yt_channel_videos WHERE video_id = 'avbZDwi4VTM';'''
cursor.execute(query)
output = cursor.fetchone()
for row in output:
  print(row)

avbZDwi4VTM
Amra Mandžukić #4 //BIH Women&#39;s National Volleyball Team
2018-05-17
10743
13
0


In [ ]:
#We see that view_count for this one is changed (10743>808) so we conclude that everything tests out as expected